<a href="https://colab.research.google.com/github/Somertonman/hackathon_2023/blob/main/data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [111]:
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

pd.options.mode.chained_assignment = None 

In [ ]:
!pip install pymorphy2
import pymorphy2

In [ ]:
!pip install transformers sacremoses
from transformers import pipeline

qa_pipeline = pipeline("question-answering", "timpal0l/mdeberta-v3-base-squad2")

In [ ]:
!wget -O dataset https://www.dropbox.com/sh/d5h5f3yrql8x392/AACQ2WYa5qYCqjC8QuVZ5TJ4a?dl=1

In [ ]:
!unzip dataset

In [ ]:
!tar -xvf  '/content/task_3/full_dataset/json.tar' -C '/content/task_3'

In [97]:
def im_pad(text):
  morph = pymorphy2.MorphAnalyzer()
  words = text.split()
  nominative_words = []
  for word in words:
    parsed_word = morph.parse(word)[0]
    nominative_word = parsed_word.normal_form
    nominative_words.append(nominative_word)
  nominative_sentence = " ".join(nominative_words)
  return nominative_sentence

In [125]:
def parse_file(file_name, num):
  df = pd.read_json(file_name).reset_index()
  
  df['id'] = num

  df = df.query('index not in ("Global_deadline", "Global_supervisor")')
  df['task_text'] = df.apply(lambda x: x['Tasks']['task_text'], axis = 1)
  df['task_responsibles_people'] = df.apply(lambda x: x['Tasks']['task_responsibles_people'], axis = 1)
  df['task_responsibles_groups'] = df.apply(lambda x: x['Tasks']['task_responsibles_groups'], axis = 1)

  question = "Какая должность?"
  df['position'] = df.apply(lambda x: im_pad(qa_pipeline(question, x['task_responsibles_people'])['answer']), axis = 1)

  question = "Какое имя?"
  df['name'] = df.apply(lambda x: im_pad(qa_pipeline(question, x['task_responsibles_people'])['answer']), axis = 1)

  df = df[['id', 'index', 'task_text', 'task_responsibles_groups', 'position', 'name']]
  return df.rename(columns={'index':'task_num', 'task_responsibles_groups':'dept'})

In [126]:
df = parse_file('/content/task_3/json/0.json', 0)

for i in range (1, 10):
  file_name = f"/content/task_3/json/{i}.json"
  df_one = parse_file(file_name)
  df = pd.concat([df, df_one])

df = df.query('name != "себя"').reset_index().drop(columns='index')

In [127]:
df

,id,task_num,task_text,dept,position,name
2,0,Task1,Закупить книги. Срок завершения: 19 апр 15.,Административно-хозяйственный отдел,специалист первый категория административно-хо...,кравцов и.е.


In [ ]:
file_name = '/content/task_3/output_decrees/json/0.json'
df = pd.read_json(file_name).reset_index()

id = re.search(r'\d*', file_name).group()
df['id'] = id
df = df.query('index not in ("Global_deadline", "Global_supervisor")')
df['task_text'] = df.apply(lambda x: x['Tasks']['task_text'], axis = 1)
df['task_responsibles_people'] = df.apply(lambda x: x['Tasks']['task_responsibles_people'], axis = 1)
df['task_responsibles_groups'] = df.apply(lambda x: x['Tasks']['task_responsibles_groups'], axis = 1)

df = df[['id', 'index', 'task_text', 'task_responsibles_people', 'task_responsibles_groups']]

question = "Какая должность?"
df['position'] = df.apply(lambda x: im_pad(qa_pipeline(question, x['task_responsibles_people'])['answer']), axis = 1)

question = "Какое имя?"
df['name'] = df.apply(lambda x: im_pad(qa_pipeline(question, x['task_responsibles_people'])['answer']), axis = 1)
df

In [101]:
df

,index,Tasks,Images,Text,id
0,Task1,{'task_text': 'Дополнить новым подпунктом 10.6...,NaN,NaN,
1,Task2,{'task_text': 'дополнить пункт 2.4.1.3 приложе...,NaN,NaN,
2,Task3,{'task_text': 'нести изменения в ОРД согласно ...,NaN,NaN,
3,Task4,{'task_text': 'пункт 2.4.1.3 изложить в следую...,NaN,NaN,
6,logo_coordinates,NaN,"[[350, 232], [590, 472]]",NaN,
7,signature_coordinates,NaN,"[[2127, 2596], [2383, 2734]]",NaN,
8,seal_coordinates,NaN,"[[2087, 2724], [2423, 3060]]",NaN,
9,header,NaN,NaN,"[[[1205, 487.0], [2385, 613.0]], page_break]",
10,name,NaN,NaN,"[[[1096, 699.0], [2385, 825.0]], page_break]",
11,intro,NaN,NaN,[page_break],


In [90]:
df['position'][8]

'Но это не значит, что мы должны быть готовы к такому повороту событий, - сказал он. - Мы должны быть готовы к тому, что, если мы не сделаем все, что в наших силах, у нас будет больше шансов на то, что мы сделаем в будущем.'

In [ ]:
en_ru_translator(qa_pipeline(question, ru_en_translator(x['task_responsibles_people'])[0]['translation_text'])[0]['translation_text'] 

In [40]:
df['task_responsibles_people'][2]

'Ответственным за исполнение настоящего приказа назначить специалиста второй категории отдела разработки систем Попова А.А.'

In [60]:
# !pip install transformers sacremoses
# from transformers import pipeline

# ru_en_translator = pipeline("translation_ru_to_en", "facebook/wmt19-ru-en")
# ner_pipeline = pipeline("ner", "dslim/bert-base-NER", aggregation_strategy="first")
qa_pipeline = pipeline("question-answering", "timpal0l/mdeberta-v3-base-squad2")
# en_ru_translator = pipeline("translation_en_to_ru", "facebook/wmt19-en-ru")

Downloading:   0%|          | 0.00/879 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/453 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [82]:
text = 'Ответственным за исполнение настоящего приказа назначить специалиста второй категории отдела разработки систем Попова А.А.'

In [61]:
question = "Какое имя?"
context = text
qa_pipeline(question, context)

{'score': 0.11700878292322159,
 'start': 110,
 'end': 122,
 'answer': ' Попова А.А.'}

In [80]:
question = "Какая должность?"
context = text
en_ru_translator(ru_en_translator(qa_pipeline(question, context)['answer'])[0]['translation_text'])[0]['translation_text']

'специалист второй категории в отделе разработки систем'

In [83]:
trans = ru_en_translator(text)
trans[0]['translation_text']

[{'translation_text': 'A specialist of the second category of the systems development department, A.A. Popov, shall be appointed responsible for the execution of this order.'}]

In [57]:
question = "What name is?"
context = trans[0]['translation_text']
en_ru_translator(qa_pipeline(question, context)['answer'])[0]['translation_text']

'Попов, А.'

In [59]:
question = "What name is?"
context = trans[0]['translation_text']
qa_pipeline(question, context)

{'score': 0.16010574996471405, 'start': 75, 'end': 85, 'answer': 'A.A. Popov'}

In [58]:
question = "Which position?"
context = trans[0]['translation_text']
en_ru_translator(qa_pipeline(question, context)['answer'])[0]['translation_text']

'Специалист второй категории отдела разработки систем'

In [91]:
def im_pad(text):
  morph = pymorphy2.MorphAnalyzer()
  words = text.split()
  nominative_words = []
  for word in words:
    parsed_word = morph.parse(word)[0]
    nominative_word = parsed_word.normal_form
    nominative_words.append(nominative_word)
  nominative_sentence = " ".join(nominative_words)
  return nominative_sentence

In [70]:
im_pad('Специалист второй категории отдела разработки систем')

'специалист второй категория отдел разработка система'